In [2]:
!pip install pysndfx

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pysndfx: filename=pysndfx-0.3.6-py3-none-any.whl size=6558 sha256=c43f48adadfbc112f7cbf3c2cd6b80c09161b8c1a18329f81b09fa894ec2193a
  Stored in directory: c:\users\napat\appdata\local\pip\cache\wheels\e8\73\04\275af1c6a789bbc12f388bce2d5353e77579e3aaadd36c61fe
Successfully built pysndfx


In [10]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
import librosa as lr
from librosa.display import specshow
from pysndfx import AudioEffectsChain

%matplotlib inline

In [ ]:
def pitch_hps(audio_samples,
              sample_rate=sr,
              window_length=4096,
              hop_length=1024,
              window=np.hanning,
              partials=5,
              plot=False):
    """Estimate the pitch contour in a monophonic audio signal."""

    f0s = []
    frequencies = np.fft.rfftfreq(window_length, 1 / sample_rate)
    window = window(window_length)
    pad = lambda a: np.pad(a, 
                           (0, window_length - len(a)),
                           mode='constant',
                           constant_values=0)
    
    # Low cut filter audio at 50 Hz.
    audio_samples = AudioEffectsChain().highpass(50)(audio_samples)

    # Go through audio frame-by-frame.
    for i in range(0, len(audio_samples), hop_length):

        # Fourier transform audio frame.

        frame = window * pad(audio_samples[i:window_length + i])
        spectrum = np.fft.rfft(frame)

        # Downsample spectrum.
        spectra = []
        for n in range(1, partials + 1):
            s = sp.signal.resample(spectrum, len(spectrum) // n)
            spectra.append(s)

        # Truncate to most downsampled spectrum.
        l = min(len(s) for s in spectra)
        a = np.zeros((len(spectra), l), dtype=spectrum.dtype)
        for i, s in enumerate(spectra):
            a[i] += s[:l]

        # Multiply spectra per frequency bin.
        hps = np.product(np.abs(a), axis=0)

        # TODO Blur spectrum to remove noise and high-frequency content .
        #kernel = sp.signal.gaussian(9, 1)
        #hps = sp.signal.fftconvolve(hps, kernel, mode='same')

        # TODO Detect peaks with a continuous wavelet transform for polyphonic signals.
        #peaks = sp.signal.find_peaks_cwt(np.abs(hps), np.arange(1, 3))

        # Pick largest peak, it's likely f0.
        peak = np.argmax(hps)
        f0 = frequencies[peak]
        f0s.append(f0)

        if plot:

            # Plot partial magnitudes individually.
            for s, ax in zip(spectra,
                             plt.subplots(len(spectra), sharex=True)[1]):
                ax.plot(np.abs(s))
            plt.suptitle('Partials')
            plt.show()

            # Plot combined spectra.
            plt.imshow(np.log(np.abs(a)), aspect='auto')
            plt.title('Spectra')
            plt.colorbar()
            plt.show()

            # Plot HPS peak.
            plt.plot(np.arange(len(hps)), np.abs(hps))
            plt.scatter(peak, np.abs(hps[peak]), color='r')
            plt.title('HPS peak')
            plt.show()
            return

    f0s = np.array(f0s)

    # Median filter out noise.
    f0s = sp.signal.medfilt(f0s, [21])

    return f0s

In [16]:
y, sr = lr.load('Piano3.mp3', sr=22050)

pitch_hps(y, plot=True)


FileNotFoundError: [WinError 2] The system cannot find the file specified

In [18]:
from pysndfx import AudioEffectsChain
import subprocess

try:
    sox_version = subprocess.run(["sox", "--version"], capture_output=True, text=True)
    print("SoX Installed:", sox_version.stdout)
except FileNotFoundError:
    print("SoX is NOT installed or not in PATH.")


SoX is NOT installed or not in PATH.
